In [ ]:
import numpy as np
import math
import os
from pathlib import Path
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import cv2
from PIL import Image

In [ ]:
# AES Algorithm Implementation
def encrypt_image_aes(key, input_image_path, output_image_path):
    # Read the input image as binary data
    with open(input_image_path, 'rb') as image_file:
        image_data = image_file.read()

    # Generate a random initialization vector (IV)
    iv = os.urandom(16)

    # Create AES cipher in CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    # Pad image data to AES block size
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(image_data) + padder.finalize()

    # Encrypt
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    # Write to output file (prepend IV)
    with open(output_image_path, 'wb') as f:
        f.write(iv + encrypted_data)

    return iv

def decrypt_image_aes(key, encrypted_image_path, output_image_path):
    # Read encrypted data
    with open(encrypted_image_path, 'rb') as f:
        encrypted_data = f.read()

    # Extract IV and ciphertext
    iv = encrypted_data[:16]
    ciphertext = encrypted_data[16:]

    # Decrypt
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()

    # Remove padding
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    decrypted_data = unpadder.update(decrypted_padded) + unpadder.finalize()

    # Write decrypted image
    with open(output_image_path, 'wb') as f:
        f.write(decrypted_data)

In [ ]:
# Unified function to calculate metrics
def calculate_metrics(original_image, encrypted_image, encrypted_image_path):
    if original_image is None or encrypted_image is None:
        print("Error: Unable to load image(s).")
        return

    # NPCR
    num_changed_pixels = np.sum(original_image != encrypted_image)
    total_pixels = original_image.size
    npcr = (num_changed_pixels / total_pixels) * 100

    # UACI
    abs_diff = np.abs(original_image - encrypted_image)
    uaci = np.mean(abs_diff / 255) * 100

    # Mean Deviation
    mean_deviation = np.mean(np.abs(original_image - encrypted_image))

    # MSE
    mse = np.mean((original_image - encrypted_image) ** 2)

    # PSNR
    max_intensity = 255
    psnr = 10 * np.log10(max_intensity ** 2 / mse)

    # Entropy
    with open(encrypted_image_path, 'rb') as f:
        encrypted_data = f.read()

    byte_frequency = [0] * 256
    for byte in encrypted_data:
        byte_frequency[byte] += 1

    entropy = 0
    total_bytes = len(encrypted_data)
    for freq in byte_frequency:
        if freq != 0:
            p = freq / total_bytes
            entropy -= p * math.log2(p)

    return npcr, uaci, mean_deviation, mse, psnr, entropy

In [ ]:
Path("AES Encryption Files").mkdir(parents=True, exist_ok=True)
input_image_path = r"Enter the path of the input image"
resized_image_path = "AES Encryption Files/original_image.png"
encrypted_image_path = "AES Encryption Files/encrypted_image.enc"
decrypted_image_path = "AES Encryption Files/decrypted_image.jpg"

# Resize image for consistency
image = Image.open(input_image_path)
original_image = image.resize((256, 256))
original_image.save(resized_image_path)

# Read as grayscale
original_image = cv2.imread(resized_image_path, cv2.IMREAD_GRAYSCALE)

# AES key (256-bit)
key = os.urandom(32)

# Encrypt and decrypt
iv = encrypt_image_aes(key, resized_image_path, encrypted_image_path)
decrypt_image_aes(key, encrypted_image_path, decrypted_image_path)

In [ ]:
# Read encrypted data as an image (approximation for pixel-based metrics)
with open(encrypted_image_path, 'rb') as f:
    encrypted_data = f.read()
encrypted_array = np.frombuffer(encrypted_data, dtype=np.uint8)[:original_image.size]
encrypted_array = encrypted_array.reshape(original_image.shape)

# Calculate metrics
npcr, uaci, mean_deviation, mse, psnr, entropy = calculate_metrics(original_image, encrypted_array, encrypted_image_path)

# Display metrics
print("NPCR score:", npcr)
print("UACI score:", uaci)
print("Mean Deviation:", mean_deviation)
print("Mean Squared Error (MSE):", mse)
print("Peak Signal-to-Noise Ratio (PSNR):", psnr)
print("Entropy of the encrypted image data:", entropy)